## Лабораторная работа №3. Классификация текстовых данных

**Выполнил:** Китайский А.С.

**Проверил:** Мохов А.С.

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd

### 1. Загрузка данных

| Вариант | Метод   | Word embedding         |
|---------|---------|------------------------|
| 8       | SVM, DT | glove-wiki-gigaword-50 |

In [3]:
categories = ['comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc'] 
remove = ('headers', 'footers', 'quotes')
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories=categories, remove=remove)

### 2. Используя GridSearchCV произведем предварительную обработку данных и настройку методов классификации в соответствие с заданием, выведем оптимальные значения параметров и результаты классификации модели (полнота, точность, f1-мера и аккуратности) с данными параметрами.

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def print_score(prediction, y_test):
    '''Расчет показателей качества'''
    precision = round(precision_score(prediction, y_test, average='weighted'), 3)
    print ('Precision score: ', precision)
    recall = round(recall_score(prediction, y_test, average='weighted'), 3)
    print ('Recall score: ', recall)
    f1 = round(f1_score(prediction, y_test, average='weighted'), 3)
    print ('F1-score: ', f1)
    accuracy = round(accuracy_score(prediction, y_test), 3)
    print ('Accuracy score: ', accuracy)

    return [precision, recall, f1, accuracy]

In [8]:
columns = ['precision', 'recall', 'f1-score', 'accuracy']
df = pd.DataFrame(columns=columns)

Метод опорных векторов (SVM):

- функция потерь (параметр kernel: ‘linear’, ‘rbf’),
- регуляризация (параметр C: {0.1, 1, 5}).

In [46]:
parameters = {'vect__max_features': [100, 500, 1000, 5000, 10000],
              'vect__stop_words': ('english', None),
              'tfidf__use_idf': (True, False),              
              'clf__kernel': ('linear', 'rbf'),
              'clf__C': (0.1, 1, 5)} 

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SVC()),])   

gs_clf = GridSearchCV(text_clf, parameters, cv=3, n_jobs=-1, verbose=10)
gs_clf.fit(twenty_train.data, twenty_train.target)

Fitting 3 folds for each of 120 candidates, totalling 360 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SVC())]),
             n_jobs=-1,
             param_grid={'clf__C': (0.1, 1, 5),
                         'clf__kernel': ('linear', 'rbf'),
                         'tfidf__use_idf': (True, False),
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': ('english', None)},
             verbose=10)

In [47]:
gs_clf.best_params_

{'clf__C': 1,
 'clf__kernel': 'linear',
 'tfidf__use_idf': True,
 'vect__max_features': 10000,
 'vect__stop_words': 'english'}

In [48]:
pred = gs_clf.predict(twenty_test.data)
df.loc['SVM_Pipeline'] = print_score(pred, twenty_test.target)

Precision score:  0.82
Recall score:  0.8
F1-score:  0.806
Accuracy score:  0.8


Дерево решений (DT):

- критерий (параметр criterion: ‘gini’, ‘entropy’),
- глубина дерева (параметр max_depth: {5, 15, 50, 100}).

In [49]:
parameters = {'vect__max_features': (100, 500, 1000, 5000, 10000),
              'vect__stop_words': ('english', None),
              'tfidf__use_idf': (True, False),              
              'clf__criterion': ('gini', 'entropy'),
              'clf__max_depth': (5, 15, 50, 100)} 

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', DecisionTreeClassifier()),])   

gs_clf = GridSearchCV(text_clf, parameters, cv=3, n_jobs=-1, verbose=10)
gs_clf.fit(twenty_train.data, twenty_train.target)

Fitting 3 folds for each of 160 candidates, totalling 480 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': (5, 15, 50, 100),
                         'tfidf__use_idf': (True, False),
                         'vect__max_features': (100, 500, 1000, 5000, 10000),
                         'vect__stop_words': ('english', None)},
             verbose=10)

In [51]:
gs_clf.best_params_

{'clf__criterion': 'gini',
 'clf__max_depth': 100,
 'tfidf__use_idf': False,
 'vect__max_features': 5000,
 'vect__stop_words': 'english'}

In [52]:
pred = gs_clf.predict(twenty_test.data)
df.loc['DT_Pipeline'] = print_score(pred, twenty_test.target)

Precision score:  0.625
Recall score:  0.64
F1-score:  0.627
Accuracy score:  0.64


In [53]:
df

,precision,recall,f1-score,accuracy
SVM_Pipeline,0.820,0.80,0.806,0.80
DT_Pipeline,0.625,0.64,0.627,0.64


### 3. Переведем выборку к векторному представлению word embedding согласно варианту.

In [54]:
import gensim.downloader
from gensim.models import KeyedVectors
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [60]:
glove_model = KeyedVectors.load_word2vec_format('glove-wiki-gigaword-50.txt', binary=False)

In [61]:
glove_model['queen']

array([ 0.37854  ,  1.8233   , -1.2648   , -0.1043   ,  0.35829  ,
        0.60029  , -0.17538  ,  0.83767  , -0.056798 , -0.75795  ,
        0.22681  ,  0.98587  ,  0.60587  , -0.31419  ,  0.28877  ,
        0.56013  , -0.77456  ,  0.071421 , -0.5741   ,  0.21342  ,
        0.57674  ,  0.3868   , -0.12574  ,  0.28012  ,  0.28135  ,
       -1.8053   , -1.0421   , -0.19255  , -0.55375  , -0.054526 ,
        1.5574   ,  0.39296  , -0.2475   ,  0.34251  ,  0.45365  ,
        0.16237  ,  0.52464  , -0.070272 , -0.83744  , -1.0326   ,
        0.45946  ,  0.25302  , -0.17837  , -0.73398  , -0.20025  ,
        0.2347   , -0.56095  , -2.2839   ,  0.0092753, -0.60284  ],
      dtype=float32)

In [62]:
glove_model.most_similar('queen') 

[('princess', 0.8515166640281677),
 ('lady', 0.8050609230995178),
 ('elizabeth', 0.7873042225837708),
 ('king', 0.7839043736457825),
 ('prince', 0.7821860909461975),
 ('coronation', 0.769277811050415),
 ('consort', 0.7626097202301025),
 ('royal', 0.7442865371704102),
 ('crown', 0.7382649779319763),
 ('victoria', 0.7285771369934082)]

In [63]:
glove_model.similarity('queen', 'king')

0.7839043

In [64]:
glove_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)

[('queen', 0.8523604273796082)]

In [70]:
def get_embedding(text_data, model=glove_model, embedding_size=50):
    
    vect = CountVectorizer(stop_words='english')
    data = vect.fit_transform(text_data)
    feature_names = vect.get_feature_names_out() 
    
    doc_vectors = [] # список векторов
    for doc in range(data.toarray().shape[0]):

        one_doc = np.zeros(embedding_size) #вектор одного документа
        cnt = 0 # счетчик слов в документе

        # итерируемся по каждому документу и проверяем присутствие слов в словаре
        for word in feature_names[(data[doc,:] > 0).toarray()[0]]:

            if word in glove_model.key_to_index.keys():
                one_doc += np.array(model[word]) # суммируем вектора каждого известного слова
                cnt += 1
        if cnt:
            one_doc = one_doc / cnt # находим средний вектор для всех слов
        doc_vectors.append(one_doc)
    return doc_vectors

doc_vectors_train = np.array(get_embedding(twenty_train.data))
doc_vectors_test = np.array(get_embedding(twenty_test.data))

In [71]:
doc_vectors_train.shape

(1554, 50)

In [72]:
doc_vectors_train[0]

array([ 0.01000642, -0.84533757,  0.60926716,  0.48792141,  0.10409343,
       -0.07573429,  0.30370214,  0.06054314,  0.15884714,  0.03314629,
        0.54265715,  0.16442856,  0.115942  ,  0.23572257, -0.05171   ,
        0.67733457, -0.22834142,  0.03366345,  0.32656929, -0.54170299,
        0.42424572, -0.84615285,  0.29271187,  0.29938671, -0.20356429,
       -0.07915413, -0.16294357, -0.04242514, -0.08724386, -0.08077858,
        0.85820499, -0.29196881,  0.42105371,  0.57581516,  0.32741285,
        0.26701228,  0.30354285,  0.34064314,  0.34719   ,  0.36802387,
        0.24394104,  0.38733857, -0.35902187,  0.27397771,  0.26723285,
        0.32777799, -0.06042128, -0.08401066,  0.12260586, -0.54703   ])

### 4. Проведем обучение и настройку тех же алгоритмов классификации и с теми же параметрами, что и в п.2, но на векторизованной выборке

Метод опорных векторов (SVM)

In [73]:
parameters = {'kernel': ('linear', 'rbf'),
              'C': (0.1, 1, 5)} 

gs_clf = GridSearchCV(SVC(), parameters, cv=5, n_jobs=-1)
gs_clf.fit(doc_vectors_train, twenty_train.target)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': (0.1, 1, 5), 'kernel': ('linear', 'rbf')})

In [74]:
gs_clf.best_params_

{'C': 5, 'kernel': 'rbf'}

In [75]:
pred = gs_clf.predict(doc_vectors_test)
df.loc['SVM_Glove'] = print_score(pred, twenty_test.target)

Precision score:  0.847
Recall score:  0.791
F1-score:  0.809
Accuracy score:  0.791


Дерево решений (DT)

In [76]:
parameters = {'criterion': ('gini', 'entropy'),
              'max_depth': (5, 15, 50, 100)} 

gs_clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5, n_jobs=-1)
gs_clf.fit(doc_vectors_train, twenty_train.target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': (5, 15, 50, 100)})

In [77]:
gs_clf.best_params_

{'criterion': 'entropy', 'max_depth': 5}

In [78]:
pred = gs_clf.predict(doc_vectors_test)
df.loc['DT_Glove'] = print_score(pred, twenty_test.target)

Precision score:  0.776
Recall score:  0.717
F1-score:  0.739
Accuracy score:  0.717


### 5. Оформим сравнительную таблицу с результатами классификации различными методами с разными настройками.

In [79]:
df

,precision,recall,f1-score,accuracy
SVM_Pipeline,0.820,0.800,0.806,0.800
DT_Pipeline,0.625,0.640,0.627,0.640
SVM_Glove,0.847,0.791,0.809,0.791
DT_Glove,0.776,0.717,0.739,0.717


**Вывод:**   
Из рассматриваемых методов наиболее подходящим для применения к данной задачи оказался метод опорных векторов с параметрами C=1 и kernal='linear'. Векторизация осуществлялась с использованием tf-idf, размерность признакого пространства составила 10000.  
В случае метода опорных векторов использование предобученной модели не дало прибавки к качеству, тогда как для дерева решений этот подход позволил улучшить score.